In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading dataset

In [ ]:
mock_df = pd.read_csv('/content/drive/MyDrive/MOCK_DATA(3).csv')

df_75 = mock_df.iloc[:int(len(mock_df) * 0.75)]
print(df_75)

     Customer ID                Country  Total Purchases  \
0           1015  Palestinian Territory               12   
1           1029                  China               47   
2           1025                Nigeria               10   
3           1006         United Kingdom               48   
4           1024            Philippines               16   
..           ...                    ...              ...   
745         1024           South Africa               16   
746         1009                Nigeria               21   
747         1012                 Angola               29   
748         1003              Indonesia               30   
749         1014               Thailand                4   

                      Sign-up Date  
0    2022-04-03 21:42:29.000000000  
1    2016-01-08 03:13:12.000000000  
2    2023-02-20 19:06:14.000000000  
3    2022-05-26 12:46:35.000000000  
4    2016-12-13 03:44:04.000000000  
..                             ...  
745  2023-06-29 16:0

Data Cleaning

In [ ]:
# To remove consecutive duplicates
mock_df_cleaned = df_75.drop_duplicates(keep = 'first')
print(mock_df_cleaned)

     Customer ID                Country  Total Purchases  \
0           1015  Palestinian Territory               12   
1           1029                  China               47   
2           1025                Nigeria               10   
3           1006         United Kingdom               48   
4           1024            Philippines               16   
..           ...                    ...              ...   
745         1024           South Africa               16   
746         1009                Nigeria               21   
747         1012                 Angola               29   
748         1003              Indonesia               30   
749         1014               Thailand                4   

                      Sign-up Date  
0    2022-04-03 21:42:29.000000000  
1    2016-01-08 03:13:12.000000000  
2    2023-02-20 19:06:14.000000000  
3    2022-05-26 12:46:35.000000000  
4    2016-12-13 03:44:04.000000000  
..                             ...  
745  2023-06-29 16:0

In [ ]:
import numpy as np

# To convert to datetime
df_75['Sign-up Date'] = pd.to_datetime(df_75['Sign-up Date'], errors='coerce')

# To replace all 2020 dates with NaN
df_75.loc[df_75['Sign-up Date'].dt.year == 2020, 'Sign-up Date'] = np.nan
print(df_75)

     Customer ID                Country  Total Purchases        Sign-up Date
0           1015  Palestinian Territory               12 2022-04-03 21:42:29
1           1029                  China               47 2016-01-08 03:13:12
2           1025                Nigeria               10 2023-02-20 19:06:14
3           1006         United Kingdom               48 2022-05-26 12:46:35
4           1024            Philippines               16 2016-12-13 03:44:04
..           ...                    ...              ...                 ...
745         1024           South Africa               16 2023-06-29 16:06:53
746         1009                Nigeria               21 2016-08-16 01:46:44
747         1012                 Angola               29 2022-10-10 22:54:07
748         1003              Indonesia               30 2015-09-17 14:03:25
749         1014               Thailand                4                 NaT

[750 rows x 4 columns]


<ipython-input-25-ce3ee012cc3e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_75['Sign-up Date'] = pd.to_datetime(df_75['Sign-up Date'], errors='coerce')


In [ ]:
# To drop rows where "Total Purchases" is less than 5, only for Canada
df_75 = df_75.drop(df_75[(df_75['Country'] == 'Canada') & (df_75['Total Purchases'] < 5)].index)

print(df_75)

     Customer ID                Country  Total Purchases        Sign-up Date
0           1015  Palestinian Territory               12 2022-04-03 21:42:29
1           1029                  China               47 2016-01-08 03:13:12
2           1025                Nigeria               10 2023-02-20 19:06:14
3           1006         United Kingdom               48 2022-05-26 12:46:35
4           1024            Philippines               16 2016-12-13 03:44:04
..           ...                    ...              ...                 ...
745         1024           South Africa               16 2023-06-29 16:06:53
746         1009                Nigeria               21 2016-08-16 01:46:44
747         1012                 Angola               29 2022-10-10 22:54:07
748         1003              Indonesia               30 2015-09-17 14:03:25
749         1014               Thailand                4                 NaT

[748 rows x 4 columns]


Feature Engineering

In [ ]:
# To calculate membership duration in years
current_year = pd.Timestamp.now().year
df_75['Years as Member'] = current_year - df_75['Sign-up Date'].dt.year
print(df_75)

     Customer ID                Country  Total Purchases        Sign-up Date  \
0           1015  Palestinian Territory               12 2022-04-03 21:42:29   
1           1029                  China               47 2016-01-08 03:13:12   
2           1025                Nigeria               10 2023-02-20 19:06:14   
3           1006         United Kingdom               48 2022-05-26 12:46:35   
4           1024            Philippines               16 2016-12-13 03:44:04   
..           ...                    ...              ...                 ...   
745         1024           South Africa               16 2023-06-29 16:06:53   
746         1009                Nigeria               21 2016-08-16 01:46:44   
747         1012                 Angola               29 2022-10-10 22:54:07   
748         1003              Indonesia               30 2015-09-17 14:03:25   
749         1014               Thailand                4                 NaT   

     Years as Member  
0               

In [ ]:
# To define the function to assign Loyalty score
def cal_loyalty(row):
  if row['Total Purchases'] > 20:
    return 3
  elif 10 <= row['Total Purchases'] <= 20:
    return 2
  elif row['Total Purchases'] < 10:
    return 2 if row['Years as Member'] > 3 else 1

# To apply function to create Loyalty score column
df_75['Loyalty Score'] = df_75.apply(cal_loyalty, axis=1)

print(df_75)

     Customer ID                Country  Total Purchases        Sign-up Date  \
0           1015  Palestinian Territory               12 2022-04-03 21:42:29   
1           1029                  China               47 2016-01-08 03:13:12   
2           1025                Nigeria               10 2023-02-20 19:06:14   
3           1006         United Kingdom               48 2022-05-26 12:46:35   
4           1024            Philippines               16 2016-12-13 03:44:04   
..           ...                    ...              ...                 ...   
745         1024           South Africa               16 2023-06-29 16:06:53   
746         1009                Nigeria               21 2016-08-16 01:46:44   
747         1012                 Angola               29 2022-10-10 22:54:07   
748         1003              Indonesia               30 2015-09-17 14:03:25   
749         1014               Thailand                4                 NaT   

     Years as Member  Loyalty Score  
0

Data Aggregation & Filtering

In [ ]:
# To exclude Mexico
df_75_filtered = df_75[df_75['Country'] != 'Mexico']
print(df_75_filtered)

     Customer ID                Country  Total Purchases        Sign-up Date  \
0           1015  Palestinian Territory               12 2022-04-03 21:42:29   
1           1029                  China               47 2016-01-08 03:13:12   
2           1025                Nigeria               10 2023-02-20 19:06:14   
3           1006         United Kingdom               48 2022-05-26 12:46:35   
4           1024            Philippines               16 2016-12-13 03:44:04   
..           ...                    ...              ...                 ...   
745         1024           South Africa               16 2023-06-29 16:06:53   
746         1009                Nigeria               21 2016-08-16 01:46:44   
747         1012                 Angola               29 2022-10-10 22:54:07   
748         1003              Indonesia               30 2015-09-17 14:03:25   
749         1014               Thailand                4                 NaT   

     Years as Member  Loyalty Score  
0

In [ ]:
# To group by country and calculate average and maximum
result = df_75_filtered.groupby('Country').agg(
    Avg_Total_Purchases=('Total Purchases', 'mean'),
    Max_Total_Purchases=('Total Purchases', 'max')
)

#To calculate most common (mode) loyalty score for each country
mode_loyalty = df_75_filtered.groupby('Country')['Loyalty Score'].agg(lambda x: x.mode()[0])

# To merge both results
result = result.join(mode_loyalty.rename('Mode_Common_Loyalty_Score'))

print(result)

             Avg_Total_Purchases  Max_Total_Purchases  \
Country                                                 
Afghanistan             8.000000                   20   
Albania                20.500000                   38   
Andorra                11.000000                   11   
Angola                 15.000000                   29   
Argentina              24.692308                   45   
...                          ...                  ...   
Vanuatu                22.000000                   22   
Venezuela              25.333333                   43   
Vietnam                24.333333                   47   
Yemen                  10.000000                   15   
Zimbabwe                8.000000                    8   

             Mode_Common_Loyalty_Score  
Country                                 
Afghanistan                          2  
Albania                              2  
Andorra                              2  
Angola                               2  
Argentina   

Data Export & Report Generation

In [ ]:
from datetime import datetime

# To get today's date in "DD_MM_YYYY" format
today_date = datetime.now().strftime("%d_%m_%Y")


In [ ]:
# To generate filename
filename = f"processed_data_{today_date}.csv"

In [ ]:
# To save DataFrame as CSV
df_75_filtered.to_csv(filename, index=False)

In [ ]:
# To shuffle and display first 10 rows (without sorting)
# Shuffle
df_75_shuffled = df_75_filtered.sample(frac=1, random_state=42)

# Display first 10 rows
print(df_75_shuffled.head(10))

     Customer ID         Country  Total Purchases        Sign-up Date  \
120         1029       Guatemala               41 2017-05-29 09:02:53   
423         1003           China               21 2016-12-22 01:52:01   
338         1025       Indonesia               18 2017-10-17 23:10:36   
354         1032  Czech Republic               41 2023-11-03 00:37:39   
419         1025         Ecuador               32 2015-05-27 19:40:51   
562         1004     Philippines               11 2019-05-09 04:46:32   
76          1014          Sweden               16 2015-10-06 10:14:42   
318         1009       Sri Lanka                1 2023-05-06 15:33:49   
250         1007         Nigeria               43 2017-09-13 23:48:37   
342         1005       Indonesia               10 2018-11-18 17:59:56   

     Years as Member  Loyalty Score  
120              8.0              3  
423              9.0              3  
338              8.0              2  
354              2.0              3  
419   

**Final Challenge**

A summarized report of my findings on this dataset based on all the steps carried out in this notebook:



Preprocessing was carried out on this dataset, which includes duplicate removal, date conversion, and handling missing values.

"Years as Member" and "Loyalty Score" columns were created based on purchase history and membership duration.

Data was filtered to exclude Mexico and aggregated by country to calculate average and maximum total purchases, and the most common loyalty score per country.

The final processed dataset was exported as a CSV file and a randomized sample of the filtered data is displayed.


In [38]:
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>